## Imports

In [19]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd

from pandas_geojson import to_geojson, write_geojson
from geopy.geocoders import GoogleV3
from textwrap import shorten

## Read-in

In [20]:
df = pd.read_csv("Proposed Projects Outside Miami-Dade County's UDB - Sheet1.csv")

## Google Maps API Geolocater Setup

In [21]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [22]:
df

,Project Name,Developer (s),Location/Address,Proposal,Acreage,Folio,story
0,Westend at Princeton,"Legacy Residential Group, CD Group and Fenix C...",14251 Southwest 256th Street in unincorporated...,This is a proposal for a 630-unit apartment co...,20-ish,30-6927-001-0010 (but please note the project ...,https://therealdeal.com/miami/2023/04/25/devel...
1,Infinity Gardens Apartments,Brandon Shpirt of BSB Global Enterprises,14505 Southwest 260th Street in unincorporated...,This is a proposal for a six-story apartment c...,13.8,30-6927-000-0320 (please note only part of the...,https://therealdeal.com/miami/2023/05/31/brand...
2,NaN,NaN,No addresses but please use folios for location,"This is a proposal for a 1,677 residential dev...",163,"30-5908-000-0030, 30-5907-000-0012, 30-5908-0...","Some background, although this is not our stor..."


In [23]:
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

df['geocoded'] = df['Location/Address'].apply(geocode)

In [24]:
df.at[2,'geocoded']=(25.665743293905635, -80.46549383357674)

In [25]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Write to GeoJSON

In [26]:
df.columns

Index(['Project Name', 'Developer (s)', 'Location/Address', 'Proposal',
       'Acreage ', 'Folio ', 'story', 'geocoded', 'lat', 'lon'],
      dtype='object')

In [27]:
geo_json = to_geojson(df=df, lat='lat', lon='lon',
                     properties=['Project Name','Developer (s)','Proposal'])
write_geojson(geo_json, filename='projects.geojson', indent=4)

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [28]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/projectsUDB'

In [29]:
print(base_name)

https://trd-digital.github.io/trd-news-interactive-maps/projectsUDB
